In [1]:
import pandas as pd
import pgaccess as pg

In [11]:
df = pg.execute_query('''
SELECT
    origin_airport_id,
    dest_airport_id,
    distance,
    EXTRACT(MONTH FROM TO_DATE(fl_date, 'YYYY-MM-DD')) AS "month",
    (crs_arr_time / 100)::int AS "hour",
    arr_delay
FROM flights
''')
df

,origin_airport_id,dest_airport_id,distance,month,arr_delay,hour
0,11193,12889,1678.0,4.0,3.0,10
1,10397,10994,259.0,4.0,-19.0,22
2,11066,13487,626.0,4.0,-1.0,8
3,13487,14831,1575.0,4.0,-2.0,21
4,10397,11481,240.0,4.0,5.0,11
...,...,...,...,...,...,...
15927480,10397,14986,444.0,4.0,-14.0,11
15927481,14986,10397,444.0,4.0,-8.0,14
15927482,11278,12892,2311.0,4.0,-1.0,20
15927483,13487,14679,1532.0,4.0,50.0,13


The features I was looking for here are the delay ratio in each category

In [43]:
origins = df.groupby('origin_airport_id').agg({'arr_delay': 'sum'})
dests = df.groupby('dest_airport_id').agg({'arr_delay': 'sum'})
ratios = pd.DataFrame({
    'origin_ratio': origins.arr_delay * 100 / origins.shape[0],
    'dest_ratio': dests.arr_delay * 100 / dests.shape[0]
})
dists = df.groupby('distance').agg(distance_ratio=('arr_delay', 'sum'))
dists = dists * 100 / dists.shape[0]
months = df.groupby('month').agg(month_ratio=('arr_delay', 'sum'))
months = months * 100 / months.shape[0]
hours = df.groupby('hour').agg(hour_ratio=('arr_delay', 'sum'))
hours = hours * 100 / hours.shape[0]

In [44]:
ratios.to_csv('../data/airport_delays.csv')
dists.to_csv('../data/distance_delays.csv')
months.to_csv('../data/month_delays.csv')
hours.to_csv('../data/hour_delays.csv')